In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture(0)
img_target = cv2.imread("test2.jpg")
temp_video = cv2.VideoCapture("test.mov")


detection = False
frame_counter = 0

success, imgVideo = temp_video.read()
hT,wT,cT = img_target.shape
imgVideo = cv2.resize(imgVideo,(wT,hT))



orb = cv2.ORB_create(nfeatures=1000)
kp1, des1 = orb.detectAndCompute(img_target,None)


while True:
    success, img_web_cam = cap.read()
    img_aug = img_web_cam.copy()
    kp2, des2 = orb.detectAndCompute(img_web_cam,None)
   
    
#     img_web_cam = cv2.drawKeypoints(img_web_cam,kp2,None) 
    
    if detection == False:
        temp_video.set(cv2.CAP_PROP_POS_FRAMES,0)
        frame_counter = 0
    else: 
        if frame_counter == temp_video.get(cv2.CAP_PROP_FRAME_COUNT):
            temp_video.set(cv2.CAP_PROP_POS_FRAMES,0)
            frame_counter = 0
        success, imgVideo = temp_video.read()
        imgVideo = cv2.resize(imgVideo,(wT,hT))
    
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
    good = []
    
    
    
    for m,n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)
            
    
    if len(good) > 20:
        detection = True
        
        srcPts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dstPts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
        
        matrix, mask = cv2.findHomography(srcPts,dstPts,cv2.RANSAC,5)
       
        
        pts = np.float32([[0,0],[0,hT],[wT,hT],[wT,0]]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,matrix)
        
        img2 = cv2.polylines(img_web_cam,[np.int32(dst)],True,(225,0,225),3)
        
        imgWrap = cv2.warpPerspective(imgVideo,matrix,(img_web_cam.shape[1],img_web_cam.shape[0]))
        
        mask_new = np.zeros((img_web_cam.shape[0],img_web_cam.shape[1]),np.uint8)
        
        cv2.fillPoly(mask_new,[np.int32(dst)],(225,225,255))
        maskInv = cv2.bitwise_not(mask_new)
        img_aug = cv2.bitwise_and(img_aug,img_aug,mask=maskInv)
        img_aug = cv2.bitwise_or(imgWrap,img_aug)
        
#         cv2.imshow("imgWrap",img_aug)
        
        cv2.fillConvexPoly(mask_new, dst.astype("int32"), (255, 255, 255),cv2.LINE_AA)
        
       
        rect = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        mask = cv2.dilate(mask_new, rect, iterations=2)
       
    
        maskScaled = mask_new.copy() / 255.0
        maskScaled = np.dstack([maskScaled] * 3)
       
    
        warpedMultiplied = cv2.multiply(imgWrap.astype("float"), maskScaled)
        imageMultiplied = cv2.multiply(img_aug.astype(float), 1.0 - maskScaled)
        output = cv2.add(warpedMultiplied, imageMultiplied)
        output = output.astype("uint8")
        
        cv2.imshow("output",output)
    
    img_features = cv2.drawMatches(img_target,kp1,img_web_cam,kp2,good,None,flags=2)
    
    
    
#     cv2.imshow("feature",img_features)
#     cv2.imshow("target_image", img_target)
    
    cv2.imshow("webcam", img_web_cam)
#     cv2.imshow("video", imgVideo)
    frame_counter += 1
    
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cv2.destroyAllWindows()
cap.release()


error: OpenCV(4.5.2) /private/var/folders/24/8k48jl6d249_n_qfxwsl6xvm0000gn/T/pip-req-build-k95y01np/opencv/modules/core/src/matmul.dispatch.cpp:531: error: (-215:Assertion failed) scn + 1 == m.cols in function 'perspectiveTransform'
